## Env

test the env for the torch

In [6]:
import torch

# Check if CUDA is available
# if torch.cuda.is_available():
#     print("CUDA is available!")
#     device = torch.device("cuda")
# else:
#     print("CUDA is not available.")
#     device = torch.device("cpu")

device = torch.device("cpu")
# Create a tensor
x = torch.rand(5, 3)

# Move the tensor to the GPU
x = x.to(device)

# Print the tensor
print(x)

tensor([[0.1517, 0.1385, 0.3776],
        [0.3969, 0.8731, 0.6285],
        [0.5430, 0.5114, 0.8157],
        [0.4969, 0.2510, 0.8303],
        [0.5157, 0.3710, 0.7567]])


## create the dataset  

In [7]:
import torch
from torch.utils.data import Dataset
from Crypto.Cipher import DES
import numpy as np


class DESDataset(Dataset):
    def __init__(self, num_samples):
        self.num_samples = num_samples
        self.key = 0x12345678.to_bytes(8, byteorder="big")

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        des = DES.new(self.key, DES.MODE_ECB)  # Initialize DES cipher with a key
        plaintext = np.random.bytes(8)  # Generate a random 64-bit plaintext
        ciphertext = des.encrypt(plaintext)  # Encrypt the plaintext
        # Convert ciphertext to binary string
        ciphertext_bin = "".join(format(byte, "08b") for byte in ciphertext)
        # Convert each bit to a uint8 and store in a numpy array
        ciphertext_bits = np.array([np.uint8(bit) for bit in ciphertext_bin])

        label = (plaintext[0] & 0x80) >> 7  # Get the first bit of the plaintext

        ciphertext_tensor = (
            torch.from_numpy(np.copy(ciphertext_bits)).float().to(device)
        )

        label_tensor = torch.tensor(label, dtype=torch.int64).float().to(device)
        return (ciphertext_tensor, label_tensor)


# if have .pth file, load it or create a new one
# try:
#     dataset = torch.load("data/des_dataset.pth")
#     print("Dataset loaded from data/des_dataset.pth")
# except FileNotFoundError:
#     print("Creating a new dataset")
#     dataset = DESDataset(10000)
#     torch.save(dataset, "data/des_dataset.pth")

dataset = DESDataset(100000)
ciphertext, lable = dataset[0]

In [8]:
# Split the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size]
)

In [9]:
import torch.nn as nn
import torch.nn.functional as F


# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(
            1, 16, 3, padding=1
        )  # Convolutional layer (16 channels, kernel size 3)
        self.pool = nn.MaxPool1d(2)  # Max pooling layer (pool size 2)
        self.fc1 = nn.Linear(32 * 16, 64)  # Fully connected layer (64 units)
        self.fc2 = nn.Linear(64, 1)  # Output layer (1 unit)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension
        x = self.pool(F.relu(self.conv1(x)))  # Convolutional layer + ReLU + pooling
        x = x.view(-1, 16 * 32)  # Flatten the tensor
        x = F.relu(self.fc1(x))  # Fully connected layer + ReLU
        x = torch.sigmoid(self.fc2(x))  # Output layer + sigmoid activation
        return x


# Initialize the CNN
net = Net()
net.to(device)

Net(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [16]:
import torch.optim as optim

# Define a Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Train the network
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')

[1,   500] loss: 132.885
[1,  1000] loss: 132.973
[2,   500] loss: 134.136
[2,  1000] loss: 133.103
[3,   500] loss: 132.553
[3,  1000] loss: 132.653
[4,   500] loss: 131.929
[4,  1000] loss: 132.926
[5,   500] loss: 132.726
[5,  1000] loss: 133.768
Finished Training


In [21]:
# test the network
correct = 0
total = 0

test_dataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

with torch.no_grad():
    for data in test_dataLoader:
        inputs, labels = data
        outputs = net(inputs)
        outputs = outputs.squeeze()
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))

Accuracy of the network on the test data: 50 %
